In [1]:
import pandas as pd
import json

def process_group_cumulative_data(file_path, output_json):
    df = pd.read_csv(file_path, encoding='ISO-8859-1', usecols=['iyear', 'gname'])

    group_mapping = {
        'ISIL': 'Islamic State of Iraq and the Levant (ISIL)',
        'taliban': 'Taliban',
        'SL': 'Shining Path (SL)'
    }

    output_data = {}

    for key, gtd_name in group_mapping.items():
        group_df = df[df['gname'] == gtd_name].copy()
        if group_df.empty:
            output_data[key] = []
            continue

        yearly_counts = group_df.groupby('iyear').size().reset_index(name='count')
        yearly_counts = yearly_counts.sort_values('iyear')
        yearly_counts['cumulative'] = yearly_counts['count'].cumsum()
        records = yearly_counts.rename(columns={'iyear': 'year', 'cumulative': 'value'})[['year', 'value']].to_dict(orient='records')
        output_data[key] = records

    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, indent=2)

process_group_cumulative_data('terrorism_dataset.csv', 'groups_temporal_cumulative.json')